In [1]:
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay

In [1]:
import dolfinx
import dolfinx.plot
import numpy as np
import ufl
from petsc4py import PETSc
from mpi4py import MPI
from math import pi, sin, cos
import pyvista

In [34]:
MAX_SUBSTEP = 10
SUBSTEP_DIV = 100

G_DIRECTION = (1,0,0)

# Function which performs loading for a particular set of parameters:
def perform_loading(L, W, NL, NW, E, kappa, y_rot, x_rot, nu, rho, g, num_steps, rtol=1e-4, atol=1e-4, max_it=10):
    
    mesh = create_mesh(L, W, NL, NW)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", el_order))

    bcs = create_bcs(mesh, V)
    
    # Compute Lame parameters:
    mu = dolfinx.Constant(mesh, E/(2*(1 + nu)))

    # Compute new body force:
    B = dolfinx.Constant(mesh, (0,0,0))
    v = ufl.TestFunction(V)
    u = dolfinx.Function(V)
    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    J = ufl.variable(ufl.det(F))
    Ic = ufl.variable(ufl.tr(C))

    # Stored strain energy density (compressible neo-Hookean model)
    psi = (mu/2)*(Ic-3) + kappa/2*(J-1)**2
    # Hyper-elasticity
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": 4}
    dx = ufl.Measure("dx", metadata=metadata)

    # Define form F (we want to find u such that F(u) = 0)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx
    
    # Delete cache of previous models:
    !rm -r /root/.cache/fenics/*
    
    problem = dolfinx.fem.NonlinearProblem(F, u, bcs)
    solver = dolfinx.NewtonSolver(MPI.COMM_WORLD, problem)
    solver.rtol = rtol
    solver.atol = atol
    solver.max_it = max_it
    
    g_vector = rho*g*np.array(G_DIRECTION)
    g_vector = rotate_gravity(g_vector, y_rot, x_rot)
    f_step = g_vector/num_steps
    for n in range(num_steps):
        print(f"Performing load step {n+1}/{num_steps}")
        for i, f_i in enumerate(f_step):
            B.value[i] = (n+1)*f_i
        num_its, converged = solver.solve(u)
        assert(converged)
        u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    volumes = compute_volume(u, mesh)
    return (u, mesh, volumes)

In [30]:
def create_mesh(L, W, NL, NW):
    mesh = dolfinx.BoxMesh(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, W, W]], [NL, NW, NW], dolfinx.cpp.mesh.CellType.hexahedron)
    return mesh

In [22]:
def create_bcs(mesh, V):
    fixed = lambda x: np.isclose(x[0], 0)
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    return bcs

In [24]:
# Using Euler angles - see https://www.autonomousrobotslab.com/frame-rotations-and-representations.html
# Here, y_rot = theta, x_rot = psi
ANGLE_TO_RAD = pi/180

def rotate_gravity(g_vector, y_rot, x_rot):
    rot_matrix = create_rot_matrix(y_rot, x_rot)
    rotated_g = rot_matrix @ g_vector
    return rotated_g

def create_rot_matrix(y_rot, x_rot):
    # NB: Negative associated with y so increasing y_rot goesin 'right direction'
    theta, psi = ANGLE_TO_RAD*y_rot, ANGLE_TO_RAD*x_rot
    theta, psi = -ANGLE_TO_RAD*y_rot, ANGLE_TO_RAD*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    return rot_matrix

def rotate_mesh(points, u, y_rot, x_rot):
    rot_matrix = create_rot_matrix(y_rot, x_rot)
    rotated_points = (rot_matrix @ points.T).T
    rotated_u = (rot_matrix @ u.T).T
    return (rotated_points, rotated_u)

In [25]:
def compute_volume(u, mesh, quad_order=4):
    before_vol, after_vol = [], []
    ndim = mesh.geometry.x.shape[1]
    I = ufl.Identity(ndim)
    dx = ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": quad_order})
    const_funspace = dolfinx.VectorFunctionSpace(mesh, ("DG", 0), dim=1)
    const_fun = dolfinx.Function(const_funspace)
    const_fun.vector[:] = np.ones(const_fun.vector[:].shape)
    ufl.inner(const_fun,const_fun)
    before_vol.append(dolfinx.fem.assemble.assemble_scalar(ufl.inner(const_fun,const_fun)*dx))
    F = I + ufl.grad(u)
    after_vol.append(dolfinx.fem.assemble.assemble_scalar(ufl.det(F)*dx))
    return (before_vol, after_vol)

In [26]:
def plot_deformation(uh, meshio_mesh, rot_y, rot_x):
    mesh = meshio_mesh
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    points, u = mesh.geometry.x, uh.compute_point_values().real
    points, u = rotate_mesh(points, u, rot_y-90, -rot_x)
    grid = pyvista.UnstructuredGrid(topology, cell_types, points)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    
    p.add_text("Deformed configuration", name="title", position="upper_edge")
    
    grid["u"] = u
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [35]:
# Material constants:
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2

# Beam dimensions:
W = 40 # in mm
L = 90 # in mm

# FEM parameters:
el_order = 2
num_steps = 50

# Store parameters in dictionary:
fixed_params = {"nu":nu, "rho":rho, "g":g, "el_order":el_order, "L":L, "W":W, "num_steps":num_steps}

# Define stiffness and beam angle to perform convergence study for:
E = 10
kappa = 1e3
rot_y = 90
rot_x = 45

# Create results dictionary:
NL, NW = 10, 10
u, mesh, volumes = perform_loading(L, W, NL, NW, E, kappa, rot_y, rot_x, nu, rho, g, num_steps)

Performing load step 1/50
Performing load step 2/50
Performing load step 3/50
Performing load step 4/50
Performing load step 5/50
Performing load step 6/50
Performing load step 7/50
Performing load step 8/50
Performing load step 9/50
Performing load step 10/50
Performing load step 11/50
Performing load step 12/50
Performing load step 13/50
Performing load step 14/50
Performing load step 15/50
Performing load step 16/50
Performing load step 17/50
Performing load step 18/50
Performing load step 19/50
Performing load step 20/50
Performing load step 21/50
Performing load step 22/50
Performing load step 23/50
Performing load step 24/50
Performing load step 25/50
Performing load step 26/50
Performing load step 27/50
Performing load step 28/50
Performing load step 29/50
Performing load step 30/50
Performing load step 31/50
Performing load step 32/50
Performing load step 33/50
Performing load step 34/50
Performing load step 35/50
Performing load step 36/50
Performing load step 37/50
Performing

In [36]:
plot_deformation(u, mesh, rot_y, rot_x)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='101...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)